# Attempt 1

* NaNs replaced by zeros
* Using QuantileTransformer on target
* Using random 80/20 train/test split
* XGBoost on RF using parameters from GridSearch

In [25]:
import numpy as np
import pandas as pd
import matplotlib
import math
from sklearn import model_selection, ensemble, metrics, linear_model, preprocessing
from matplotlib import pyplot as plt
%matplotlib inline

In [48]:
data = pd.read_csv('dataset/undp_train.csv', na_values='nan')

In [49]:
test = pd.read_csv('dataset/undp_test.csv', na_values='nan')

In [32]:
data = data.set_index(['year', 'country'])
test = test.set_index(['year', 'country'])

In [39]:
full = data.append(test)

In [44]:
full.to_csv('dataset/full.csv', na_rep='nan')

In [45]:
full = pd.read_csv('dataset/full.csv', na_values='nan')

In [38]:
# FOR ONE HOT ENCODING
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder

X_train = data.iloc[:, 0:-1].values
X_test = test.values
y_train = data.iloc[:, -1].values

# imputing missing values
# simple imputer
imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
imputer.fit(full.drop(columns='y').values)
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

# OneHotEncoder
enc = OneHotEncoder(sparse=False)
print(enc.fit_transform(X[:, 1].reshape(-1, 1)).shape)
enc.fit(full.drop(columns='y').values.reshape(-1,1))

X_train = np.hstack
X = np.hstack((X, enc.fit_transform(X[:, 1].reshape(-1, 1))))
X = np.delete(X, 1, 1)

#normalize
scaler = preprocessing.StandardScaler()
X[:, :75] = scaler.fit_transform(X[:, :75])

#creating a target transformer (normal)
qt = QuantileTransformer(n_quantiles=1000, output_distribution='normal' )

(4755, 429)


In [ ]:
X_test 

In [13]:
#XGBoost
import xgboost as xgb
XGB = TransTargReg(xgb.XGBRegressor(
    max_depth=5,
    learning_rate=0.1,
    n_estimators=1500,
    verbosity=1,
    silent=None,
    objective='reg:squarederror',
    booster='gbtree',
    n_jobs=3,
    nthread=None,
    gamma=0,
    min_child_weight=1,
    max_delta_step=0,
    subsample=1,
    colsample_bytree=1,
    colsample_bylevel=1,
    colsample_bynode=0.3,
    reg_alpha=0,
    reg_lambda=0.11,
    scale_pos_weight=1,
    base_score=0.5,
    random_state=42,
    seed=None,
    missing=None,
    importance_type='gain'),
                   qt)
XGB.fit(X_train, y_train)

print(MSE(y_train, XGB.predict(X_train))**0.5)
print(MSE(y_test, XGB.predict(X_test))**0.5)

0.0009545862826864125
0.012114620334841404
